In [411]:
from gurobipy import *
import pandas as pd
import numpy as np

### Problem 4

In [412]:
# Read excel sheets and transform them into lists and matrices
demand_info=pd.read_excel('OR109-1_case01_data.xlsx','Demand',index_col = 0)
initial_inventory_info=pd.read_excel('OR109-1_case01_data.xlsx','Initial inventory',index_col = 0)
shipping_cost_info=pd.read_excel('OR109-1_case01_data.xlsx','Shipping cost',index_col = 0)
intransit=pd.read_excel('OR109-1_case01_data.xlsx','In-transit',index_col = 0)
size_info=pd.read_excel('OR109-1_case01_data.xlsx','Size',index_col = 0)
price_and_cost_info=pd.read_excel('OR109-1_case01_data.xlsx','Price and cost',index_col = 0)
shortage_info=pd.read_excel('OR109-1_case01_data.xlsx','Shortage',index_col = 0)

In [413]:
size=size_info['Cubic meter'].values.tolist()

In [414]:
backorder=shortage_info['Backorder'].values.tolist()
lost_sales=shortage_info['Lost sales'].values.tolist()
backorder_percentage=shortage_info['Backorder percentage'].values.tolist()

In [415]:
backorder

[535, 250, 1345, 980, 345, 1575, 705, 820, 1650, 930]

In [416]:
lost_sales

[5700, 3000, 17900, 10600, 4900, 22500, 7100, 11400, 24000, 11600]

In [417]:
backorder_percentage

[0.0, 0.7, 0.1, 1.0, 1.0, 0.3, 0.6, 0.2, 0.1, 0.5]

In [418]:
demand=demand_info.values.tolist()
initial_inventory=initial_inventory_info['Initial inventory'].values.tolist()
express_delivery_cost=shipping_cost_info['Express delivery'].values.tolist()
air_freight_cost=shipping_cost_info['Air freight'].values.tolist()
ocean_freight_cost=[0]*10

In [419]:
purchasing_cost=price_and_cost_info['Purchasing cost'].values.tolist()
holding_cost=price_and_cost_info['Holding'].values.tolist()
fix_cost=[100,80,50]

In [420]:
shipping_cost=[express_delivery_cost,air_freight_cost,ocean_freight_cost]

In [421]:
intransit.insert(loc=2, column='6', value=0)
intransit.insert(loc=3, column='7', value=0)
intransit.insert(loc=4, column='8', value=0)
intransit.insert(loc=0, column='3', value=0)
intransit=intransit.values.tolist()

In [422]:
product_len=range(10)
month_len=range(6)
shipping_method_len=range(3)

In [423]:
cs1 = Model("cs1")

In [424]:
# add variables as a list
# 定義X ijk 並根據k 來直接產生t (例如說：express delievery 直接將k+1=t 為 month t is available)
x_method1_K = []
for i in product_len:
    x_method1_K.append([])
    for k in month_len:
        x_method1_K[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method1 K" +  str(i+1) +'_'+ str(k+1)))

x_method1_T= []
for i in product_len:
    x_method1_T.append([])
    for k in month_len:
        x_method1_T[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method1 T" +  str(i+1) +'_'+ str(k+1)))
        

In [425]:
x_method2_K = []
for i in product_len:
    x_method2_K.append([])
    for k in month_len:
        x_method2_K[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method2 K" +  str(i+1) +'_'+ str(k+1)))
        
x_method2_T = []
for i in product_len:
    x_method2_T.append([])
    for k in month_len:
        x_method2_T[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method2 T" +  str(i+1) +'_'+ str(k+1)))

In [426]:
x_method3_K = []
for i in product_len:
    x_method3_K.append([])
    for k in month_len:
        x_method3_K[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method3 K" +  str(i+1) +'_'+ str(k+1)))
        
x_method3_T = []
for i in product_len:
    x_method3_T.append([])
    for k in month_len:
        x_method3_T[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_method3 T" +  str(i+1) +'_'+ str(k+1)))

In [427]:
#定義 y 
y=[]
for i in product_len:
    y.append([])
    for k in month_len:
        y[i].append(cs1.addVar( vtype = GRB.CONTINUOUS, name = "y" +  str(i+1) +'_'+ str(k+1)))

In [428]:
#定義 z 
z=[]
for j in shipping_method_len:
    z.append([])
    for k in month_len:
        z[j].append(cs1.addVar(lb = 0, vtype = GRB.BINARY, name = "z" +  str(j+1) +'_'+ str(k+1)))

In [429]:
#定義 w
w=[]
for k in month_len:
    w.append(cs1.addVar(lb = 0, vtype = GRB.INTEGER , name = "w" +  str(k+1)))

In [430]:
#定義 u
u=[]
for i in product_len:
    u.append([])
    for k in month_len:
        u[i].append(cs1.addVar( vtype = GRB.CONTINUOUS, name = "u" +  str(i+1) +'_'+ str(k+1)))

In [431]:
#定義 r
r=[]
for i in product_len:
    r.append([])
    for k in month_len:
        r[i].append(cs1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "r" +  str(i+1) +'_'+ str(k+1)))

In [432]:
#定義 q
q=[]
for i in product_len:
    q.append([])
    for k in month_len:
        q[i].append(cs1.addVar(lb = 0,  vtype = GRB.CONTINUOUS, name = "q" +  str(i+1) +'_'+ str(k+1)))

In [433]:
#定義 ypn
ypn=[]
for i in product_len:
    ypn.append([])
    for k in month_len:
        ypn[i].append(cs1.addVar(lb = 0, vtype = GRB.BINARY, name = "ypn" +  str(i+1) +'_'+ str(k+1)))

In [434]:
# setting the objective function 
cs1.setObjective(
    
    quicksum(quicksum(purchasing_cost[i]*x_method1_K[i][k] for i in product_len) for k in month_len)+  # P*x的部分
    quicksum(quicksum(purchasing_cost[i]*x_method2_K[i][k] for i in product_len) for k in month_len)+
    quicksum(quicksum(purchasing_cost[i]*x_method3_K[i][k] for i in product_len) for k in month_len)+ 
    
    quicksum(quicksum(holding_cost[i]*r[i][k] for i in product_len) for k in month_len)+ # y的部分
    
    quicksum(quicksum(fix_cost[j]*z[j][k] for j in shipping_method_len) for k in month_len)+  # z的部分
    
    quicksum(quicksum(shipping_cost[0][i]*x_method1_K[i][k] for i in product_len) for k in month_len)+ # C*x的部分
    quicksum(quicksum(shipping_cost[1][i]*x_method2_K[i][k] for i in product_len) for k in month_len)+
    quicksum(quicksum(shipping_cost[2][i]*x_method3_K[i][k] for i in product_len) for k in month_len)+ 
    
    quicksum( 2750*w[k] for k in month_len)+ # w的部分
    
    quicksum( quicksum(backorder[i]*backorder_percentage[i]*(-u[i][k]) for i in product_len) for k in month_len)+ # BA的部分
                 
    quicksum( quicksum(lost_sales[i]*(1-backorder_percentage[i])*(-u[i][k]) for i in product_len) for k in month_len) # Si(1 − Ai)(−uit)的部分
    
    , GRB.MINIMIZE) 

In [435]:
cs1.addConstrs( ( (ypn[i][k]-0.5)*y[i][k]  >=0
                for i in product_len
                for k in month_len 
                ), "decide ypn" )

{(0, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 0): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 0): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 0): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 0): <gurobi

In [436]:
cs1.addConstrs( ((ypn[i][k] == 1) >> ( q[i][k]  == 1)
                for i in product_len
                for k in month_len )
                , "q == 1" )

{(0, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 0): <gurobi.GenConstr *Awaiting Model Upd

In [437]:
cs1.addConstrs( ((ypn[i][k] == 0) >> ( q[i][k]  == backorder_percentage[i])
                for i in product_len
                for k in month_len)
                , "q == A" )

{(0, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (0, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (1, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 0): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 1): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 2): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 3): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 4): <gurobi.GenConstr *Awaiting Model Update*>,
 (2, 5): <gurobi.GenConstr *Awaiting Model Update*>,
 (3, 0): <gurobi.GenConstr *Awaiting Model Upd

In [444]:
cs1.addConstrs( ( y[i][k]  <=  (q[i][k] * y[i][k])
                for i in product_len
                for k in month_len 
                ), "y q" )

{(0, 0): <gurobi.QConstr Not Yet Added>,
 (0, 1): <gurobi.QConstr Not Yet Added>,
 (0, 2): <gurobi.QConstr Not Yet Added>,
 (0, 3): <gurobi.QConstr Not Yet Added>,
 (0, 4): <gurobi.QConstr Not Yet Added>,
 (0, 5): <gurobi.QConstr Not Yet Added>,
 (1, 0): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (2, 0): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (3, 0): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi.QConstr Not Yet Added>,
 (3, 5): <gurobi.QConstr Not Yet Added>,
 (4, 0): <gurobi

In [391]:
cs1.addConstrs( ( u[i][k]  <= 0 
                for i in product_len
                for k in month_len 
                ), "min 1" )
cs1.addConstrs( ( u[i][k]  <= y[i][k] 
                for i in product_len
                for k in month_len 
                ), "min 2" )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


In [392]:
cs1.addConstrs( ( r[i][k]  >= 0 
                for i in product_len
                for k in month_len 
                ), "max 1" )
cs1.addConstrs( ( r[i][k]  >=  y[i][k] 
                for i in product_len
                for k in month_len 
                ), "max 2" )

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


In [393]:
cs1.addConstrs( ( (1/30)*quicksum( size[i]*x_method3_K[i][k] for i in product_len)  <= w[k] 
                for k in month_len 
                ), "container costs" )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [394]:
cs1.addConstrs( ( y[i][k] ==  initial_inventory[i]  + x_method1_T[i][k] + x_method2_T[i][k]+ x_method3_T[i][k] + 
                 intransit[i][k]
                 - demand[i][k] 
                for k in month_len 
                for i in product_len
                if k==0), "initial inventory" )


cs1.addConstrs( ( y[i][k] ==  y[i][k-1]*q[i][k-1]  + x_method1_T[i][k] + x_method2_T[i][k]+ x_method3_T[i][k] + 
                 intransit[i][k]
                 - demand[i][k] 
                for k in month_len 
                for i in product_len
                if k!=0), "inventory" )


{(1, 0): <gurobi.QConstr Not Yet Added>,
 (1, 1): <gurobi.QConstr Not Yet Added>,
 (1, 2): <gurobi.QConstr Not Yet Added>,
 (1, 3): <gurobi.QConstr Not Yet Added>,
 (1, 4): <gurobi.QConstr Not Yet Added>,
 (1, 5): <gurobi.QConstr Not Yet Added>,
 (1, 6): <gurobi.QConstr Not Yet Added>,
 (1, 7): <gurobi.QConstr Not Yet Added>,
 (1, 8): <gurobi.QConstr Not Yet Added>,
 (1, 9): <gurobi.QConstr Not Yet Added>,
 (2, 0): <gurobi.QConstr Not Yet Added>,
 (2, 1): <gurobi.QConstr Not Yet Added>,
 (2, 2): <gurobi.QConstr Not Yet Added>,
 (2, 3): <gurobi.QConstr Not Yet Added>,
 (2, 4): <gurobi.QConstr Not Yet Added>,
 (2, 5): <gurobi.QConstr Not Yet Added>,
 (2, 6): <gurobi.QConstr Not Yet Added>,
 (2, 7): <gurobi.QConstr Not Yet Added>,
 (2, 8): <gurobi.QConstr Not Yet Added>,
 (2, 9): <gurobi.QConstr Not Yet Added>,
 (3, 0): <gurobi.QConstr Not Yet Added>,
 (3, 1): <gurobi.QConstr Not Yet Added>,
 (3, 2): <gurobi.QConstr Not Yet Added>,
 (3, 3): <gurobi.QConstr Not Yet Added>,
 (3, 4): <gurobi

In [395]:
# add constraints and name them
cs1.addConstrs( (quicksum(x_method1_K[i][k] for i in product_len)  <= 7274 * z[0][k] 
                 for k in month_len 
                ), "binary z 1 " )
cs1.addConstrs( (quicksum(x_method2_K[i][k] for i in product_len)  <= 7274 * z[1][k] 
                 for k in month_len 
                ), "binary z 2" )
cs1.addConstrs( (quicksum(x_method3_K[i][k] for i in product_len)  <= 7274 * z[2][k] 
                 for k in month_len 
                ), "binary z 3" )

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>}

In [396]:
# add constraints and name them
cs1.addConstrs( ( x_method1_K[i][k] ==  x_method1_T[i][k+1]
                for k in range(5) 
                for i in product_len
                ), "method 1 T = K " )

cs1.addConstrs(  (x_method2_K[i][k] ==  x_method2_T[i][k+2]
                for k in range(4) 
                for i in product_len
                ), "method 2 T = K " )

cs1.addConstrs(  (x_method3_K[i][k] ==  x_method3_T[i][k+3]
                for k in range(3) 
                for i in product_len
                ), "method 3 T = K " )

cs1.addConstrs(  (x_method1_T[i][k]==0
                for k in range(1) 
                for i in product_len
                ), "method 1  T==0 " )
cs1.addConstrs(  (x_method2_T[i][k]==0
                for k in range(2) 
                for i in product_len
                ), "method 2  T==0 " )

cs1.addConstrs(  (x_method3_T[i][k]==0
                for k in range(3) 
                for i in product_len
                ), "method 3  T==0 " )


cs1.addConstrs(  (x_method1_K[i][k+5]==0
                for k in range(1) 
                for i in product_len
                ), "method 1  K==0 " )
cs1.addConstrs(  (x_method2_K[i][k+4]==0
                for k in range(2) 
                for i in product_len
                ), "method 2  K==0 " )

cs1.addConstrs(  (x_method3_K[i][k+3]==0
                for k in range(3) 
                for i in product_len
                ), "method 3  K==0 " )


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 7): <gurobi.Constr *Awaiting Model Update*>,
 (1, 8): <gurobi.Constr *Awaiting Model Update*>,
 (1, 9): <gurobi.Constr *Awaiting Model Update*>,


In [397]:
cs1.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 514 rows, 684 columns and 1024 nonzeros
Model fingerprint: 0x1cd81e1a
Model has 170 quadratic constraints
Model has 120 general constraints
Variable types: 600 continuous, 84 integer (78 binary)
Coefficient statistics:
  Matrix range     [2e-04, 7e+03]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [5e-01, 1e+00]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 7e+02]
  QRHS range       [1e+00, 2e+02]
Presolve removed 473 rows and 505 columns
Presolve time: 0.01s
Presolved: 105 rows, 198 columns, 488 nonzeros
Variable types: 165 continuous, 33 integer (30 binary)

Root relaxation: objective 1.650623e+07, 30 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

### 檢視結果

In [398]:
y

[[<gurobi.Var y1_1 (value 662.0)>,
  <gurobi.Var y1_2 (value 607.0)>,
  <gurobi.Var y1_3 (value 435.0)>,
  <gurobi.Var y1_4 (value 241.0)>,
  <gurobi.Var y1_5 (value 147.0)>,
  <gurobi.Var y1_6 (value 0.0)>],
 [<gurobi.Var y2_1 (value 410.0)>,
  <gurobi.Var y2_2 (value 357.0)>,
  <gurobi.Var y2_3 (value 289.0)>,
  <gurobi.Var y2_4 (value 104.0)>,
  <gurobi.Var y2_5 (value 91.0)>,
  <gurobi.Var y2_6 (value 0.0)>],
 [<gurobi.Var y3_1 (value 346.0)>,
  <gurobi.Var y3_2 (value 167.0)>,
  <gurobi.Var y3_3 (value 166.0)>,
  <gurobi.Var y3_4 (value 117.0)>,
  <gurobi.Var y3_5 (value 0.0)>,
  <gurobi.Var y3_6 (value 0.0)>],
 [<gurobi.Var y4_1 (value 208.0)>,
  <gurobi.Var y4_2 (value 258.0)>,
  <gurobi.Var y4_3 (value 180.0)>,
  <gurobi.Var y4_4 (value 49.0)>,
  <gurobi.Var y4_5 (value 0.0)>,
  <gurobi.Var y4_6 (value 0.0)>],
 [<gurobi.Var y5_1 (value 365.0)>,
  <gurobi.Var y5_2 (value 303.0)>,
  <gurobi.Var y5_3 (value 220.0)>,
  <gurobi.Var y5_4 (value 130.0)>,
  <gurobi.Var y5_5 (value 0.0)

In [399]:
x_method1_K

[[<gurobi.Var x_method1 K1_1 (value 0.0)>,
  <gurobi.Var x_method1 K1_2 (value 0.0)>,
  <gurobi.Var x_method1 K1_3 (value 0.0)>,
  <gurobi.Var x_method1 K1_4 (value 0.0)>,
  <gurobi.Var x_method1 K1_5 (value 0.0)>,
  <gurobi.Var x_method1 K1_6 (value 0.0)>],
 [<gurobi.Var x_method1 K2_1 (value 0.0)>,
  <gurobi.Var x_method1 K2_2 (value 0.0)>,
  <gurobi.Var x_method1 K2_3 (value 0.0)>,
  <gurobi.Var x_method1 K2_4 (value 0.0)>,
  <gurobi.Var x_method1 K2_5 (value 0.0)>,
  <gurobi.Var x_method1 K2_6 (value 0.0)>],
 [<gurobi.Var x_method1 K3_1 (value 0.0)>,
  <gurobi.Var x_method1 K3_2 (value 0.0)>,
  <gurobi.Var x_method1 K3_3 (value 0.0)>,
  <gurobi.Var x_method1 K3_4 (value 0.0)>,
  <gurobi.Var x_method1 K3_5 (value 0.0)>,
  <gurobi.Var x_method1 K3_6 (value 0.0)>],
 [<gurobi.Var x_method1 K4_1 (value 0.0)>,
  <gurobi.Var x_method1 K4_2 (value 0.0)>,
  <gurobi.Var x_method1 K4_3 (value 0.0)>,
  <gurobi.Var x_method1 K4_4 (value 0.0)>,
  <gurobi.Var x_method1 K4_5 (value 0.0)>,
  <gurob

In [400]:
x_method2_K

[[<gurobi.Var x_method2 K1_1 (value 0.0)>,
  <gurobi.Var x_method2 K1_2 (value 0.0)>,
  <gurobi.Var x_method2 K1_3 (value 0.0)>,
  <gurobi.Var x_method2 K1_4 (value 0.0)>,
  <gurobi.Var x_method2 K1_5 (value 0.0)>,
  <gurobi.Var x_method2 K1_6 (value 0.0)>],
 [<gurobi.Var x_method2 K2_1 (value 0.0)>,
  <gurobi.Var x_method2 K2_2 (value 0.0)>,
  <gurobi.Var x_method2 K2_3 (value 0.0)>,
  <gurobi.Var x_method2 K2_4 (value 0.0)>,
  <gurobi.Var x_method2 K2_5 (value 0.0)>,
  <gurobi.Var x_method2 K2_6 (value 0.0)>],
 [<gurobi.Var x_method2 K3_1 (value 0.0)>,
  <gurobi.Var x_method2 K3_2 (value 0.0)>,
  <gurobi.Var x_method2 K3_3 (value 0.0)>,
  <gurobi.Var x_method2 K3_4 (value 0.0)>,
  <gurobi.Var x_method2 K3_5 (value 0.0)>,
  <gurobi.Var x_method2 K3_6 (value 0.0)>],
 [<gurobi.Var x_method2 K4_1 (value 0.0)>,
  <gurobi.Var x_method2 K4_2 (value 0.0)>,
  <gurobi.Var x_method2 K4_3 (value 0.0)>,
  <gurobi.Var x_method2 K4_4 (value 0.0)>,
  <gurobi.Var x_method2 K4_5 (value 0.0)>,
  <gurob

In [401]:
x_method3_K

[[<gurobi.Var x_method3 K1_1 (value 0.0)>,
  <gurobi.Var x_method3 K1_2 (value 0.0)>,
  <gurobi.Var x_method3 K1_3 (value 38.0)>,
  <gurobi.Var x_method3 K1_4 (value 0.0)>,
  <gurobi.Var x_method3 K1_5 (value 0.0)>,
  <gurobi.Var x_method3 K1_6 (value 0.0)>],
 [<gurobi.Var x_method3 K2_1 (value 0.0)>,
  <gurobi.Var x_method3 K2_2 (value 0.0)>,
  <gurobi.Var x_method3 K2_3 (value 45.0)>,
  <gurobi.Var x_method3 K2_4 (value 0.0)>,
  <gurobi.Var x_method3 K2_5 (value 0.0)>,
  <gurobi.Var x_method3 K2_6 (value 0.0)>],
 [<gurobi.Var x_method3 K3_1 (value 0.0)>,
  <gurobi.Var x_method3 K3_2 (value 82.0)>,
  <gurobi.Var x_method3 K3_3 (value 200.0)>,
  <gurobi.Var x_method3 K3_4 (value 0.0)>,
  <gurobi.Var x_method3 K3_5 (value 0.0)>,
  <gurobi.Var x_method3 K3_6 (value 0.0)>],
 [<gurobi.Var x_method3 K4_1 (value 0.0)>,
  <gurobi.Var x_method3 K4_2 (value 97.0)>,
  <gurobi.Var x_method3 K4_3 (value 155.0)>,
  <gurobi.Var x_method3 K4_4 (value 0.0)>,
  <gurobi.Var x_method3 K4_5 (value 0.0)>,


In [402]:
x_method1_T

[[<gurobi.Var x_method1 T1_1 (value 0.0)>,
  <gurobi.Var x_method1 T1_2 (value 0.0)>,
  <gurobi.Var x_method1 T1_3 (value 0.0)>,
  <gurobi.Var x_method1 T1_4 (value 0.0)>,
  <gurobi.Var x_method1 T1_5 (value 0.0)>,
  <gurobi.Var x_method1 T1_6 (value 0.0)>],
 [<gurobi.Var x_method1 T2_1 (value 0.0)>,
  <gurobi.Var x_method1 T2_2 (value 0.0)>,
  <gurobi.Var x_method1 T2_3 (value 0.0)>,
  <gurobi.Var x_method1 T2_4 (value 0.0)>,
  <gurobi.Var x_method1 T2_5 (value 0.0)>,
  <gurobi.Var x_method1 T2_6 (value 0.0)>],
 [<gurobi.Var x_method1 T3_1 (value 0.0)>,
  <gurobi.Var x_method1 T3_2 (value 0.0)>,
  <gurobi.Var x_method1 T3_3 (value 0.0)>,
  <gurobi.Var x_method1 T3_4 (value 0.0)>,
  <gurobi.Var x_method1 T3_5 (value 0.0)>,
  <gurobi.Var x_method1 T3_6 (value 0.0)>],
 [<gurobi.Var x_method1 T4_1 (value 0.0)>,
  <gurobi.Var x_method1 T4_2 (value 0.0)>,
  <gurobi.Var x_method1 T4_3 (value 0.0)>,
  <gurobi.Var x_method1 T4_4 (value 0.0)>,
  <gurobi.Var x_method1 T4_5 (value 0.0)>,
  <gurob

In [403]:
x_method2_T

[[<gurobi.Var x_method2 T1_1 (value 0.0)>,
  <gurobi.Var x_method2 T1_2 (value 0.0)>,
  <gurobi.Var x_method2 T1_3 (value 0.0)>,
  <gurobi.Var x_method2 T1_4 (value 0.0)>,
  <gurobi.Var x_method2 T1_5 (value 0.0)>,
  <gurobi.Var x_method2 T1_6 (value 0.0)>],
 [<gurobi.Var x_method2 T2_1 (value 0.0)>,
  <gurobi.Var x_method2 T2_2 (value 0.0)>,
  <gurobi.Var x_method2 T2_3 (value 0.0)>,
  <gurobi.Var x_method2 T2_4 (value 0.0)>,
  <gurobi.Var x_method2 T2_5 (value 0.0)>,
  <gurobi.Var x_method2 T2_6 (value 0.0)>],
 [<gurobi.Var x_method2 T3_1 (value 0.0)>,
  <gurobi.Var x_method2 T3_2 (value 0.0)>,
  <gurobi.Var x_method2 T3_3 (value 0.0)>,
  <gurobi.Var x_method2 T3_4 (value 0.0)>,
  <gurobi.Var x_method2 T3_5 (value 0.0)>,
  <gurobi.Var x_method2 T3_6 (value 0.0)>],
 [<gurobi.Var x_method2 T4_1 (value 0.0)>,
  <gurobi.Var x_method2 T4_2 (value 0.0)>,
  <gurobi.Var x_method2 T4_3 (value 0.0)>,
  <gurobi.Var x_method2 T4_4 (value 0.0)>,
  <gurobi.Var x_method2 T4_5 (value 0.0)>,
  <gurob

In [404]:
x_method3_T

[[<gurobi.Var x_method3 T1_1 (value 0.0)>,
  <gurobi.Var x_method3 T1_2 (value 0.0)>,
  <gurobi.Var x_method3 T1_3 (value 0.0)>,
  <gurobi.Var x_method3 T1_4 (value 0.0)>,
  <gurobi.Var x_method3 T1_5 (value 0.0)>,
  <gurobi.Var x_method3 T1_6 (value 38.0)>],
 [<gurobi.Var x_method3 T2_1 (value 0.0)>,
  <gurobi.Var x_method3 T2_2 (value 0.0)>,
  <gurobi.Var x_method3 T2_3 (value 0.0)>,
  <gurobi.Var x_method3 T2_4 (value 0.0)>,
  <gurobi.Var x_method3 T2_5 (value 0.0)>,
  <gurobi.Var x_method3 T2_6 (value 45.0)>],
 [<gurobi.Var x_method3 T3_1 (value 0.0)>,
  <gurobi.Var x_method3 T3_2 (value 0.0)>,
  <gurobi.Var x_method3 T3_3 (value 0.0)>,
  <gurobi.Var x_method3 T3_4 (value 0.0)>,
  <gurobi.Var x_method3 T3_5 (value 82.0)>,
  <gurobi.Var x_method3 T3_6 (value 200.0)>],
 [<gurobi.Var x_method3 T4_1 (value 0.0)>,
  <gurobi.Var x_method3 T4_2 (value 0.0)>,
  <gurobi.Var x_method3 T4_3 (value 0.0)>,
  <gurobi.Var x_method3 T4_4 (value 0.0)>,
  <gurobi.Var x_method3 T4_5 (value 97.0)>,
  

In [405]:
z

[[<gurobi.Var z1_1 (value 1.0)>,
  <gurobi.Var z1_2 (value -0.0)>,
  <gurobi.Var z1_3 (value -0.0)>,
  <gurobi.Var z1_4 (value -0.0)>,
  <gurobi.Var z1_5 (value -0.0)>,
  <gurobi.Var z1_6 (value 0.0)>],
 [<gurobi.Var z2_1 (value 1.0)>,
  <gurobi.Var z2_2 (value -0.0)>,
  <gurobi.Var z2_3 (value -0.0)>,
  <gurobi.Var z2_4 (value -0.0)>,
  <gurobi.Var z2_5 (value 0.0)>,
  <gurobi.Var z2_6 (value 0.0)>],
 [<gurobi.Var z3_1 (value 1.0)>,
  <gurobi.Var z3_2 (value 1.0)>,
  <gurobi.Var z3_3 (value 1.0)>,
  <gurobi.Var z3_4 (value 0.0)>,
  <gurobi.Var z3_5 (value 0.0)>,
  <gurobi.Var z3_6 (value 0.0)>]]

In [406]:
w

[<gurobi.Var w1 (value 1.0)>,
 <gurobi.Var w2 (value 2.0)>,
 <gurobi.Var w3 (value 3.0)>,
 <gurobi.Var w4 (value -0.0)>,
 <gurobi.Var w5 (value -0.0)>,
 <gurobi.Var w6 (value -0.0)>]

In [407]:
u

[[<gurobi.Var u1_1 (value 0.0)>,
  <gurobi.Var u1_2 (value 0.0)>,
  <gurobi.Var u1_3 (value 0.0)>,
  <gurobi.Var u1_4 (value 0.0)>,
  <gurobi.Var u1_5 (value 0.0)>,
  <gurobi.Var u1_6 (value 0.0)>],
 [<gurobi.Var u2_1 (value 0.0)>,
  <gurobi.Var u2_2 (value 0.0)>,
  <gurobi.Var u2_3 (value 0.0)>,
  <gurobi.Var u2_4 (value 0.0)>,
  <gurobi.Var u2_5 (value 0.0)>,
  <gurobi.Var u2_6 (value 0.0)>],
 [<gurobi.Var u3_1 (value 0.0)>,
  <gurobi.Var u3_2 (value 0.0)>,
  <gurobi.Var u3_3 (value 0.0)>,
  <gurobi.Var u3_4 (value 0.0)>,
  <gurobi.Var u3_5 (value 0.0)>,
  <gurobi.Var u3_6 (value 0.0)>],
 [<gurobi.Var u4_1 (value 0.0)>,
  <gurobi.Var u4_2 (value 0.0)>,
  <gurobi.Var u4_3 (value 0.0)>,
  <gurobi.Var u4_4 (value 0.0)>,
  <gurobi.Var u4_5 (value 0.0)>,
  <gurobi.Var u4_6 (value 0.0)>],
 [<gurobi.Var u5_1 (value 0.0)>,
  <gurobi.Var u5_2 (value 0.0)>,
  <gurobi.Var u5_3 (value 0.0)>,
  <gurobi.Var u5_4 (value 0.0)>,
  <gurobi.Var u5_5 (value 0.0)>,
  <gurobi.Var u5_6 (value 0.0)>],
 [<gu

In [408]:
r

[[<gurobi.Var r1_1 (value 662.0)>,
  <gurobi.Var r1_2 (value 607.0)>,
  <gurobi.Var r1_3 (value 435.0)>,
  <gurobi.Var r1_4 (value 241.0)>,
  <gurobi.Var r1_5 (value 147.0)>,
  <gurobi.Var r1_6 (value 0.0)>],
 [<gurobi.Var r2_1 (value 410.0)>,
  <gurobi.Var r2_2 (value 357.0)>,
  <gurobi.Var r2_3 (value 289.0)>,
  <gurobi.Var r2_4 (value 104.0)>,
  <gurobi.Var r2_5 (value 91.0)>,
  <gurobi.Var r2_6 (value 0.0)>],
 [<gurobi.Var r3_1 (value 346.0)>,
  <gurobi.Var r3_2 (value 167.0)>,
  <gurobi.Var r3_3 (value 166.0)>,
  <gurobi.Var r3_4 (value 117.0)>,
  <gurobi.Var r3_5 (value 0.0)>,
  <gurobi.Var r3_6 (value 0.0)>],
 [<gurobi.Var r4_1 (value 208.0)>,
  <gurobi.Var r4_2 (value 258.0)>,
  <gurobi.Var r4_3 (value 180.0)>,
  <gurobi.Var r4_4 (value 49.0)>,
  <gurobi.Var r4_5 (value 0.0)>,
  <gurobi.Var r4_6 (value 0.0)>],
 [<gurobi.Var r5_1 (value 365.0)>,
  <gurobi.Var r5_2 (value 303.0)>,
  <gurobi.Var r5_3 (value 220.0)>,
  <gurobi.Var r5_4 (value 130.0)>,
  <gurobi.Var r5_5 (value 0.0)

In [409]:
q

[[<gurobi.Var q1_1 (value 1.0)>,
  <gurobi.Var q1_2 (value 1.0)>,
  <gurobi.Var q1_3 (value 1.0)>,
  <gurobi.Var q1_4 (value 1.0)>,
  <gurobi.Var q1_5 (value 1.0)>,
  <gurobi.Var q1_6 (value 1.0)>],
 [<gurobi.Var q2_1 (value 1.0)>,
  <gurobi.Var q2_2 (value 1.0)>,
  <gurobi.Var q2_3 (value 1.0)>,
  <gurobi.Var q2_4 (value 1.0)>,
  <gurobi.Var q2_5 (value 1.0)>,
  <gurobi.Var q2_6 (value 1.0)>],
 [<gurobi.Var q3_1 (value 1.0)>,
  <gurobi.Var q3_2 (value 1.0)>,
  <gurobi.Var q3_3 (value 1.0)>,
  <gurobi.Var q3_4 (value 1.0)>,
  <gurobi.Var q3_5 (value 0.1)>,
  <gurobi.Var q3_6 (value 1.0)>],
 [<gurobi.Var q4_1 (value 1.0)>,
  <gurobi.Var q4_2 (value 1.0)>,
  <gurobi.Var q4_3 (value 1.0)>,
  <gurobi.Var q4_4 (value 1.0)>,
  <gurobi.Var q4_5 (value 1.0)>,
  <gurobi.Var q4_6 (value 1.0)>],
 [<gurobi.Var q5_1 (value 1.0)>,
  <gurobi.Var q5_2 (value 1.0)>,
  <gurobi.Var q5_3 (value 1.0)>,
  <gurobi.Var q5_4 (value 1.0)>,
  <gurobi.Var q5_5 (value 1.0)>,
  <gurobi.Var q5_6 (value 1.0)>],
 [<gu

In [410]:
ypn

[[<gurobi.Var ypn1_1 (value 1.0)>,
  <gurobi.Var ypn1_2 (value 1.0)>,
  <gurobi.Var ypn1_3 (value 1.0)>,
  <gurobi.Var ypn1_4 (value 1.0)>,
  <gurobi.Var ypn1_5 (value 1.0)>,
  <gurobi.Var ypn1_6 (value 1.0)>],
 [<gurobi.Var ypn2_1 (value 1.0)>,
  <gurobi.Var ypn2_2 (value 1.0)>,
  <gurobi.Var ypn2_3 (value 1.0)>,
  <gurobi.Var ypn2_4 (value 1.0)>,
  <gurobi.Var ypn2_5 (value 1.0)>,
  <gurobi.Var ypn2_6 (value 1.0)>],
 [<gurobi.Var ypn3_1 (value 1.0)>,
  <gurobi.Var ypn3_2 (value 1.0)>,
  <gurobi.Var ypn3_3 (value 1.0)>,
  <gurobi.Var ypn3_4 (value 1.0)>,
  <gurobi.Var ypn3_5 (value -0.0)>,
  <gurobi.Var ypn3_6 (value 1.0)>],
 [<gurobi.Var ypn4_1 (value 1.0)>,
  <gurobi.Var ypn4_2 (value 1.0)>,
  <gurobi.Var ypn4_3 (value 1.0)>,
  <gurobi.Var ypn4_4 (value 1.0)>,
  <gurobi.Var ypn4_5 (value 1.0)>,
  <gurobi.Var ypn4_6 (value 1.0)>],
 [<gurobi.Var ypn5_1 (value 1.0)>,
  <gurobi.Var ypn5_2 (value 1.0)>,
  <gurobi.Var ypn5_3 (value 1.0)>,
  <gurobi.Var ypn5_4 (value 1.0)>,
  <gurobi.Var y